In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.cluster import KMeans
from sklearn.metrics.cluster import adjusted_rand_score
from matplotlib import pyplot as plt
from sklearn.decomposition import PCA

all_df = pd.read_csv('/Users/mario/code/pb_speaker_adaptation/src/analysis/adaptive_speak_eval_22june/adaptive_speak_eval_all_v15.csv', on_bad_lines='skip')
print('target domain   :', all_df['target domain'].unique())
print('listener domain :', all_df['listener domain'].unique())
print('simulator domain:', all_df['simulator domain'].unique())

df = pd.read_csv('/Users/mario/code/pb_speaker_adaptation/src/analysis/adaptive_speak_eval_22june/adaptive_speak_eval_food_v2.csv')
print('target domain   :', df['target domain'].unique())
print('listener domain :', df['listener domain'].unique())
print('simulator domain:', df['simulator domain'].unique())

In [2]:
_df = pd.concat([
    pd.read_csv('/Users/mario/code/pb_speaker_adaptation/src/analysis/28sept/adaptive_speak_test_out_domain_appliances.csv', error_bad_lines=False, warn_bad_lines=True),
    pd.read_csv('/Users/mario/code/pb_speaker_adaptation/src/analysis/28sept/adaptive_speak_test_out_domain_food.csv', error_bad_lines=False, warn_bad_lines=True),
    pd.read_csv('/Users/mario/code/pb_speaker_adaptation/src/analysis/28sept/adaptive_speak_test_out_domain_indoor.csv', error_bad_lines=False, warn_bad_lines=True),
    pd.read_csv('/Users/mario/code/pb_speaker_adaptation/src/analysis/28sept/adaptive_speak_test_out_domain_outdoor.csv', error_bad_lines=False, warn_bad_lines=True),
    pd.read_csv('/Users/mario/code/pb_speaker_adaptation/src/analysis/28sept/adaptive_speak_test_out_domain_vehicles.csv', error_bad_lines=False, warn_bad_lines=True)
])

# pd.to_numeric(df['target domain'], errors='coerce')

print('target domain   :', _df['target domain'].unique())
print('listener domain :', _df['listener domain'].unique())
print('simulator domain:', _df['interpreter domain'].unique())
_df = _df.dropna()
print('Data points:', len(_df))

id_df = _df[(_df['target domain'] == _df['listener domain'])]
ood_df = _df[(_df['target domain'] != _df['listener domain'])]

df = ood_df
df.head(2)

/var/folders/0s/847v13g104z23kr193w_v2640000gn/T/ipykernel_16528/3501453246.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  pd.read_csv('/Users/mario/code/pb_speaker_adaptation/src/analysis/28sept/adaptive_speak_test_out_domain_appliances.csv', error_bad_lines=False, warn_bad_lines=True),
/var/folders/0s/847v13g104z23kr193w_v2640000gn/T/ipykernel_16528/3501453246.py:2: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  pd.read_csv('/Users/mario/code/pb_speaker_adaptation/src/analysis/28sept/adaptive_speak_test_out_domain_appliances.csv', error_bad_lines=False, warn_bad_lines=True),
b'Skipping line 1397: expected 198 fields, saw 443\n'
/var/folders/0s/847v13g104z23kr193w_v2640000gn/T/ipykernel_16528/3501453246.py:2: DtypeWarning: Columns (54,55,56,64,65,66) have mixed types. Specify dtype option on imp

target domain   : ['food' 'vehicles' 'outdoor' 'indoor' 'appliances']
listener domain : ['indoor' 'appliances' 'food' 'outdoor' 'vehicles']
simulator domain: ['indoor' 'appliances' 'food' 'outdoor' 'vehicles']
Data points: 5817


,Unnamed: 0,target domain,listener domain,interpreter domain,target img idx,img path #0,img path #1,img path #2,img path #3,img path #4,...,int_list_acc10,int_list_acc11,int_list_acc12,int_list_acc13,int_list_acc14,int_list_acc15,int_list_acc16,int_list_acc17,int_list_acc18,int_list_acc19
2,2,food,indoor,indoor,0,/home/brandizzi/pb_speaker_adaptation/dataset/...,/home/brandizzi/pb_speaker_adaptation/dataset/...,/home/brandizzi/pb_speaker_adaptation/dataset/...,/home/brandizzi/pb_speaker_adaptation/dataset/...,/home/brandizzi/pb_speaker_adaptation/dataset/...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,5,food,indoor,indoor,2,/home/brandizzi/pb_speaker_adaptation/dataset/...,/home/brandizzi/pb_speaker_adaptation/dataset/...,/home/brandizzi/pb_speaker_adaptation/dataset/...,/home/brandizzi/pb_speaker_adaptation/dataset/...,/home/brandizzi/pb_speaker_adaptation/dataset/...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
N_STEPS = 20
print('Representations: how many change?')
for n in range(N_STEPS):
    print(f's{n}:', len(df[df['original h0'] != df[f'adapted h0 s{n}']]), '/', len(df))

print('\nUtterances: how many change?')
for n in range(N_STEPS):
#     print(f's{n}:', len(df[df['original utt'] != df[f'adapted utt s{n}']]), '/', len(df))
    if n > 0:
        print(f's{n-1} -> s{n}:', len(df[df[f'adapted utt s{n}'] != df[f'adapted utt s{n-1}']]))
    else:
        print(f'orig -> s{n}:', len(df[df[f'adapted utt s{n}'] != df['original utt']]))

Representations: how many change?
s0: 4758 / 4758
s1: 4758 / 4758
s2: 4758 / 4758
s3: 4758 / 4758
s4: 4758 / 4758
s5: 4758 / 4758
s6: 4758 / 4758
s7: 4758 / 4758
s8: 4758 / 4758
s9: 4758 / 4758
s10: 4758 / 4758
s11: 4758 / 4758
s12: 4758 / 4758
s13: 4758 / 4758
s14: 4758 / 4758
s15: 4758 / 4758
s16: 4758 / 4758
s17: 4758 / 4758
s18: 4758 / 4758
s19: 4758 / 4758

Utterances: how many change?
orig -> s0: 4758
s0 -> s1: 1324
s1 -> s2: 1665
s2 -> s3: 2019
s3 -> s4: 2107
s4 -> s5: 2113
s5 -> s6: 2160
s6 -> s7: 2132
s7 -> s8: 2043
s8 -> s9: 1902
s9 -> s10: 1860
s10 -> s11: 1666
s11 -> s12: 1573
s12 -> s13: 1406
s13 -> s14: 1278
s14 -> s15: 1172
s15 -> s16: 1011
s16 -> s17: 980
s17 -> s18: 906
s18 -> s19: 859


## Diagnostic classification

In [4]:
CLF_TYPE = 'logistic'


In [5]:
def train_and_classify(X, Y, classifier_type='LR'):
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, stratify=Y, test_size=0.3, random_state=1)
    # print(f'X_train: {X_train.shape}  X_test: {X_test.shape}  Y_train: {Y_train.shape}  Y_test: {Y_test.shape}')

    scaler = preprocessing.StandardScaler().fit(X_train)
    X_train_scaled = scaler.transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    if classifier_type == 'MLP':
        clf = MLPClassifier(random_state=1, max_iter=10000).fit(X_train_scaled, Y_train)
    elif classifier_type == 'logistic':
        clf = LogisticRegression(random_state=1, max_iter=10000).fit(X_train_scaled, Y_train)
    # elif classifier_type == 'linear':
    #     clf = LinearRegression().fit(X_train_scaled, Y_train)
    else:
        raise ValueError()

    Y_pred = clf.predict(X_test_scaled)

    return classification_report(Y_test, Y_pred), clf, scaler

In [6]:
def classify(X, Y, classifier, scaler):
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, stratify=Y, test_size=0.3, random_state=1)
    # print(f'X_train: {X_train.shape}  X_test: {X_test.shape}  Y_train: {Y_train.shape}  Y_test: {Y_test.shape}')

    X_train_scaled = scaler.transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    Y_pred = classifier.predict(X_test_scaled)

    return classification_report(Y_test, Y_pred)

# Is the domain of the target image encoded in $h_0$?

### Predict target domain from non-adapted hidden representation

In [7]:
X, Y = [], []

for _, row in df.iterrows():
    X.append(eval(row['original h0']))
    Y.append(row['target domain'])

X, Y = np.array(X), np.array(Y)

report, target_classifier, target_scaler = train_and_classify(X, Y, CLF_TYPE)
print(report)

              precision    recall  f1-score   support

  appliances       0.98      0.97      0.98       143
        food       0.99      0.99      0.99       198
      indoor       0.99      0.99      0.99       325
     outdoor       0.99      1.00      1.00       556
    vehicles       1.00      0.99      1.00       206

    accuracy                           0.99      1428
   macro avg       0.99      0.99      0.99      1428
weighted avg       0.99      0.99      0.99      1428



### Predict _target_ domain from adapted hidden representation (probe trained on non-adapted)

In [8]:
for step in np.arange(N_STEPS):

    print(f'Step {step}')

    X, Y = [], []

    for _, row in df.iterrows():
        X.append(eval(row[f'adapted h0 s{step}']))
        Y.append(row['target domain'])

    X, Y = np.array(X), np.array(Y)

    print(classify(X, Y, target_classifier, target_scaler))

#     pca = PCA(2)
#     scaler = preprocessing.StandardScaler().fit(X)
#     X_2dim = pca.fit_transform(scaler.transform(X))

#     for l in df['target domain'].unique():
#         plt.scatter(X_2dim[Y == l, 0], X_2dim[Y == l, 1], label=l)
#     plt.legend()
#     plt.show()

Step 0
              precision    recall  f1-score   support

  appliances       0.72      0.48      0.58       143
        food       0.61      0.56      0.59       198
      indoor       0.65      0.63      0.64       325
     outdoor       0.68      0.61      0.64       556
    vehicles       0.33      0.55      0.41       206

    accuracy                           0.59      1428
   macro avg       0.60      0.57      0.57      1428
weighted avg       0.62      0.59      0.59      1428

Step 1
              precision    recall  f1-score   support

  appliances       0.44      0.25      0.32       143
        food       0.31      0.28      0.29       198
      indoor       0.43      0.42      0.43       325
     outdoor       0.59      0.47      0.53       556
    vehicles       0.23      0.45      0.31       206

    accuracy                           0.41      1428
   macro avg       0.40      0.38      0.37      1428
weighted avg       0.45      0.41      0.42      1428

Step 2
 

              precision    recall  f1-score   support

  appliances       0.11      0.06      0.07       143
        food       0.17      0.17      0.17       198
      indoor       0.21      0.20      0.21       325
     outdoor       0.44      0.26      0.33       556
    vehicles       0.15      0.38      0.22       206

    accuracy                           0.23      1428
   macro avg       0.22      0.21      0.20      1428
weighted avg       0.28      0.23      0.24      1428

Step 18
              precision    recall  f1-score   support

  appliances       0.10      0.05      0.07       143
        food       0.17      0.17      0.17       198
      indoor       0.21      0.21      0.21       325
     outdoor       0.44      0.26      0.33       556
    vehicles       0.15      0.38      0.22       206

    accuracy                           0.23      1428
   macro avg       0.22      0.21      0.20      1428
weighted avg       0.28      0.23      0.24      1428

Step 19
      

### Predict _target_ domain from adapted hidden representation (probe retrained at each step)

In [9]:
for step in np.arange(N_STEPS):

    print(f'Step {step}')

    X, Y = [], []

    for _, row in df.iterrows():
        X.append(eval(row[f'adapted h0 s{step}']))
        Y.append(row['target domain'])

    X, Y = np.array(X), np.array(Y)

    report, _, _ = train_and_classify(X, Y, CLF_TYPE)
    print(report)
    
#     pca = PCA(2)
#     scaler = preprocessing.StandardScaler().fit(X)
#     X_2dim = pca.fit_transform(scaler.transform(X))

#     for l in df['target domain'].unique():
#         plt.scatter(X_2dim[Y == l, 0], X_2dim[Y == l, 1], label=l)
#     plt.legend()
#     plt.show()

Step 0
              precision    recall  f1-score   support

  appliances       0.99      0.95      0.97       143
        food       0.99      0.98      0.98       198
      indoor       0.98      0.98      0.98       325
     outdoor       0.98      0.99      0.98       556
    vehicles       0.98      0.97      0.97       206

    accuracy                           0.98      1428
   macro avg       0.98      0.97      0.98      1428
weighted avg       0.98      0.98      0.98      1428

Step 1
              precision    recall  f1-score   support

  appliances       0.96      0.93      0.95       143
        food       0.99      0.98      0.98       198
      indoor       0.98      0.97      0.97       325
     outdoor       0.97      0.98      0.98       556
    vehicles       0.96      0.96      0.96       206

    accuracy                           0.97      1428
   macro avg       0.97      0.96      0.97      1428
weighted avg       0.97      0.97      0.97      1428

Step 2
 

              precision    recall  f1-score   support

  appliances       0.65      0.71      0.68       143
        food       0.84      0.82      0.83       198
      indoor       0.71      0.72      0.71       325
     outdoor       0.81      0.77      0.79       556
    vehicles       0.74      0.79      0.76       206

    accuracy                           0.76      1428
   macro avg       0.75      0.76      0.76      1428
weighted avg       0.77      0.76      0.76      1428

Step 18
              precision    recall  f1-score   support

  appliances       0.65      0.70      0.68       143
        food       0.85      0.82      0.83       198
      indoor       0.70      0.72      0.71       325
     outdoor       0.81      0.77      0.79       556
    vehicles       0.74      0.80      0.76       206

    accuracy                           0.76      1428
   macro avg       0.75      0.76      0.75      1428
weighted avg       0.76      0.76      0.76      1428

Step 19
      

# Is the domain of the listener encoded in $h_0$?

### Try predicting _listener_ domain from non-adapted hidden representation _using the image domain probing classifier_.  

That is, is the listener domain encoded similarly to the target domain?

In [10]:
X, Y = [], []

for _, row in df.iterrows():
    X.append(eval(row['original h0']))
    Y.append(row['listener domain'])

X, Y = np.array(X), np.array(Y)


print(classify(X, Y, target_classifier, target_scaler))

              precision    recall  f1-score   support

  appliances       0.00      0.00      0.00     216.0
        food       0.00      0.00      0.00     315.0
      indoor       0.00      0.00      0.00     416.0
     outdoor       0.00      0.00      0.00     216.0
    vehicles       0.00      0.00      0.00     265.0

    accuracy                           0.00    1428.0
   macro avg       0.00      0.00      0.00    1428.0
weighted avg       0.00      0.00      0.00    1428.0



### Predict _listener_ domain from non-adapted hidden representation _using a listener domain probing classifier_.  

In [11]:
X, Y = [], []

for _, row in df.iterrows():
    X.append(eval(row['original h0']))
    Y.append(row['listener domain'])

X, Y = np.array(X), np.array(Y)


report, listener_classifier, listener_scaler = train_and_classify(X, Y, CLF_TYPE)
print(report)


              precision    recall  f1-score   support

  appliances       0.33      0.40      0.36       216
        food       0.40      0.39      0.39       315
      indoor       0.54      0.58      0.56       416
     outdoor       0.39      0.32      0.35       216
    vehicles       0.30      0.26      0.28       265

    accuracy                           0.41      1428
   macro avg       0.39      0.39      0.39      1428
weighted avg       0.41      0.41      0.41      1428



### Predict _listener_ domain from adapted hidden representation (probe trained on non-adapted)

In [12]:
for step in np.arange(N_STEPS):

    print(f'Step {step}')

    X, Y = [], []
    for _, row in df.iterrows():
        X.append(eval(row[f'adapted h0 s{step}']))
        Y.append(row['listener domain'])

    X, Y = np.array(X), np.array(Y)
    
    print(classify(X, Y, listener_classifier, listener_scaler))
#     print(train_and_classify(X, Y, CLF_TYPE))

Step 0
              precision    recall  f1-score   support

  appliances       0.11      0.17      0.13       216
        food       0.27      0.29      0.28       315
      indoor       0.33      0.25      0.29       416
     outdoor       0.15      0.14      0.14       216
    vehicles       0.17      0.17      0.17       265

    accuracy                           0.21      1428
   macro avg       0.21      0.20      0.20      1428
weighted avg       0.23      0.21      0.22      1428

Step 1
              precision    recall  f1-score   support

  appliances       0.09      0.15      0.12       216
        food       0.24      0.28      0.26       315
      indoor       0.27      0.19      0.22       416
     outdoor       0.14      0.12      0.13       216
    vehicles       0.15      0.13      0.14       265

    accuracy                           0.18      1428
   macro avg       0.18      0.18      0.17      1428
weighted avg       0.20      0.18      0.19      1428

Step 2
 

              precision    recall  f1-score   support

  appliances       0.13      0.25      0.17       216
        food       0.20      0.25      0.22       315
      indoor       0.21      0.11      0.15       416
     outdoor       0.13      0.13      0.13       216
    vehicles       0.20      0.13      0.16       265

    accuracy                           0.17      1428
   macro avg       0.17      0.18      0.17      1428
weighted avg       0.18      0.17      0.17      1428

Step 18
              precision    recall  f1-score   support

  appliances       0.12      0.24      0.16       216
        food       0.20      0.25      0.22       315
      indoor       0.21      0.12      0.15       416
     outdoor       0.13      0.13      0.13       216
    vehicles       0.20      0.13      0.15       265

    accuracy                           0.17      1428
   macro avg       0.17      0.17      0.17      1428
weighted avg       0.18      0.17      0.17      1428

Step 19
      

### Predict _listener_ domain from adapted hidden representation (probe retrained at each step)

In [13]:
for step in np.arange(N_STEPS):

    print(f'Step {step}')

    X, Y = [], []
    for _, row in df.iterrows():
        X.append(eval(row[f'adapted h0 s{step}']))
        Y.append(row['listener domain'])

    X, Y = np.array(X), np.array(Y)
    
    report, _, _ = train_and_classify(X, Y, CLF_TYPE)
    print(report)

Step 0
              precision    recall  f1-score   support

  appliances       0.55      0.61      0.58       216
        food       0.58      0.54      0.56       315
      indoor       0.72      0.79      0.75       416
     outdoor       0.63      0.55      0.59       216
    vehicles       0.55      0.52      0.54       265

    accuracy                           0.62      1428
   macro avg       0.61      0.60      0.60      1428
weighted avg       0.62      0.62      0.62      1428

Step 1
              precision    recall  f1-score   support

  appliances       0.60      0.67      0.63       216
        food       0.66      0.60      0.63       315
      indoor       0.76      0.82      0.79       416
     outdoor       0.65      0.64      0.65       216
    vehicles       0.65      0.58      0.62       265

    accuracy                           0.68      1428
   macro avg       0.66      0.66      0.66      1428
weighted avg       0.68      0.68      0.68      1428

Step 2
 

              precision    recall  f1-score   support

  appliances       0.87      0.90      0.89       216
        food       0.90      0.88      0.89       315
      indoor       0.91      0.96      0.93       416
     outdoor       0.90      0.88      0.89       216
    vehicles       0.91      0.85      0.88       265

    accuracy                           0.90      1428
   macro avg       0.90      0.89      0.89      1428
weighted avg       0.90      0.90      0.90      1428

Step 18
              precision    recall  f1-score   support

  appliances       0.87      0.91      0.89       216
        food       0.90      0.87      0.88       315
      indoor       0.90      0.96      0.93       416
     outdoor       0.92      0.87      0.89       216
    vehicles       0.89      0.85      0.87       265

    accuracy                           0.90      1428
   macro avg       0.90      0.89      0.89      1428
weighted avg       0.90      0.90      0.90      1428

Step 19
      

# Is the adaptation itself (described as the difference between adapted and non-adapted $h_0$) predictive of listener domains?

### Predict listener domain from difference between adapted and non-adapted hidden representation (probe retrained at each step)

In [14]:
for step in np.arange(N_STEPS):

    print(f'Step {step}')

    X, Y = [], []
    for _, row in df.iterrows():
        X.append(np.array(eval(row[f'adapted h0 s{step}'])) - np.array(eval(row['original h0'])))
        Y.append(row['listener domain'])

    X, Y = np.array(X), np.array(Y)

    report, _, _ = train_and_classify(X, Y, CLF_TYPE)
    print(report)

Step 0
              precision    recall  f1-score   support

  appliances       0.55      0.58      0.57       216
        food       0.61      0.51      0.56       315
      indoor       0.67      0.76      0.71       416
     outdoor       0.56      0.56      0.56       216
    vehicles       0.62      0.58      0.60       265

    accuracy                           0.61      1428
   macro avg       0.60      0.60      0.60      1428
weighted avg       0.61      0.61      0.61      1428

Step 1
              precision    recall  f1-score   support

  appliances       0.64      0.69      0.66       216
        food       0.70      0.62      0.65       315
      indoor       0.73      0.78      0.76       416
     outdoor       0.61      0.66      0.64       216
    vehicles       0.66      0.59      0.62       265

    accuracy                           0.68      1428
   macro avg       0.67      0.67      0.66      1428
weighted avg       0.68      0.68      0.68      1428

Step 2
 

              precision    recall  f1-score   support

  appliances       0.89      0.89      0.89       216
        food       0.92      0.87      0.89       315
      indoor       0.88      0.96      0.92       416
     outdoor       0.91      0.88      0.90       216
    vehicles       0.89      0.86      0.87       265

    accuracy                           0.90      1428
   macro avg       0.90      0.89      0.90      1428
weighted avg       0.90      0.90      0.90      1428

Step 18
              precision    recall  f1-score   support

  appliances       0.89      0.90      0.89       216
        food       0.92      0.88      0.90       315
      indoor       0.89      0.96      0.92       416
     outdoor       0.91      0.89      0.90       216
    vehicles       0.90      0.85      0.87       265

    accuracy                           0.90      1428
   macro avg       0.90      0.89      0.90      1428
weighted avg       0.90      0.90      0.90      1428

Step 19
      

## Diagnostic clustering

### Cluster non-adapted hidden representations (vs. target domains)

### Cluster adapted hidden representations (vs. target domains)



### Cluster adapted hidden representations (vs. listener domains)


### Cluster difference between adapted and non-adapted hidden representations (vs. listener domains)
